In [1]:
from skimage.metrics import structural_similarity as ssim
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import pandas as pd
import shutil
import cv2, glob, random, math, numpy as np, dlib, itertools

In [2]:
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
cwd = os.getcwd()

df = pd.read_csv(cwd + "/data/legend.csv")

new_df = pd.DataFrame(columns=['image', 'emotion'])

for idx, row in df.iterrows():
    new_df = pd.concat([new_df, pd.DataFrame.from_records([{'image': row.image, 'emotion': row.emotion}])])

for emo in dictionary:
    img_list = os.listdir(cwd + "/images/" + emo)

    for i in range(len(img_list)):
        path = emo + "/" + img_list[i]

        new_df = pd.concat([new_df, pd.DataFrame.from_records([{'image': path, 'emotion': emo}])])

new_df.to_csv(cwd + "/data_csv/all_data.csv", index=False)

In [3]:
# read and write all data
cwd = os.getcwd()
df = pd.read_csv(cwd + "/data_csv/all_data.csv")

# change emotion label to same base
df["emotion"].replace({"anger": "ANGER", "contempt": "CONTEMPT", "disgust": "DISGUST", "fear": "FEAR", \
                        "happiness": "HAPPINESS", "neutral": "NEUTRAL", "sadness": "SADNESS", "surprise": "SURPRISE"}, inplace=True)

# replace string emotion label with integer
df['emotion'].replace({'ANGER': 0, 'CONTEMPT': 1, 'DISGUST': 2, 'FEAR': 3, \
                        'HAPPINESS': 4,  'NEUTRAL': 5, 'SADNESS': 6, 'SURPRISE': 7}, inplace=True)

In [4]:
df

,image,emotion
0,facial-expressions_2868588k.jpg,0
1,facial-expressions_2868585k.jpg,7
2,facial-expressions_2868584k.jpg,2
3,facial-expressions_2868582k.jpg,3
4,Aaron_Eckhart_0001.jpg,5
...,...,...
80095,SURPRISE/surprise (995).png,7
80096,SURPRISE/surprise (996).jpg,7
80097,SURPRISE/surprise (996).png,7
80098,SURPRISE/surprise (998).png,7


In [5]:
df.groupby('emotion').count()

,image
emotion,
0,10794
1,2832
2,5009
3,5911
4,17852
5,17166
6,11069
7,9467


In [6]:
# create folder
dir_img = cwd + '/cleaned_images'
if os.path.exists(dir_img):
    shutil.rmtree(dir_img)
os.makedirs(dir_img)

dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
for emo in dictionary:
    os.makedirs(dir_img + "/" + emo)

all_image = []

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(cwd + "/predictor/shape_predictor_68_face_landmarks.dat")

# set size
width = 256
height = 256
dim = (width, height)

for idx, row in df.iterrows():
    imagePath = cwd + "/images/" + row.image

    image = cv2.imread(imagePath)

    # resize
    image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)

    # change color
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # check blurry
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    if fm < 10:
        continue
    
    # detect face with haarcascade
    face_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_frontalface_alt.xml')
    eye_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_eye_tree_eyeglasses.xml')
    smile_cascade = cv2.CascadeClassifier('haarcascade/haarcascade_smile.xml')

    face = face_cascade.detectMultiScale(
        gray,
        scaleFactor = 1.1,
        minNeighbors = 4,
        minSize = (200, 200),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    
    if len(face) != 1:
        continue

    for (x, y, w, h) in face:
        roi_gray = gray[y:y+h, x:x+w]

    smile = smile_cascade.detectMultiScale(
        roi_gray,
        scaleFactor = 1.16,
        minNeighbors = 35,
        minSize = (25, 25),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    
    if len(smile) < 1:
        continue

    eyes = eye_cascade.detectMultiScale(roi_gray)

    if len(eyes) < 2:
        continue
    
    # detect face with shape predictor
    rects = detector(image, 0)

    if len(rects) == 0:
        continue

    # collect preprocessed data
    all_image.append([row.image, row.emotion])

    # write preprocessed image 
    cv2.imwrite(os.path.join(dir_img, row.image), gray)

In [7]:
len(all_image)

24113

In [8]:
def mse(imageA, imageB):
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])
	
	# return the MSE, the lower the error, the more "similar"
	return err

In [9]:
'''
dictionary = ['ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS',  'NEUTRAL', 'SADNESS', 'SURPRISE']
cwd = os.getcwd()

delete = []

for emo in range(3, 5):
    print(dictionary[emo])

    img_list = os.listdir(cwd + "/cleaned_images/" + dictionary[emo])

    # same scale
    scale_percent = 100

    for i in range(len(img_list)):
        if not os.path.exists("cleaned_images/" + dictionary[emo] + "/" + img_list[i]):
            continue

        img1 = cv2.imread("cleaned_images/" + dictionary[emo] + "/" + img_list[i])
        
        # set size
        width = int(img1.shape[1] * scale_percent / 100)
        height = int(img1.shape[0] * scale_percent / 100)
        dim = (width, height)
        
        # resize for comparation
        img1 = cv2.resize(img1, dim, interpolation = cv2.INTER_AREA)

        # convert the images to grayscale
        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
        
        for j in range(i+1, len(img_list)):
            # do not run same image
            if img_list[i] == img_list[j]:
                continue
            
            # do not run deleted image
            if not os.path.exists("cleaned_images/" + dictionary[emo] + "/" + img_list[j]):
                continue

            img2 = cv2.imread("cleaned_images/" + dictionary[emo] + "/" + img_list[j])

            # resize for comparation
            img2 = cv2.resize(img2, dim, interpolation = cv2.INTER_AREA)

            # convert the images to grayscale
            img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

            # compare the images
            m = mse(img1, img2)

            if m == 0 and os.path.exists("images/" + dictionary[emo] + "/" + img_list[j]):
                print(img_list[i], img_list[j])
                os.remove("images/" + dictionary[emo] + "/" + img_list[j])
                delete.append([dictionary[emo] + "/" + img_list[j], dictionary[emo]])'''

'\ndictionary = [\'ANGER\', \'CONTEMPT\', \'DISGUST\', \'FEAR\', \'HAPPINESS\',  \'NEUTRAL\', \'SADNESS\', \'SURPRISE\']\ncwd = os.getcwd()\n\ndelete = []\n\nfor emo in range(3, 5):\n    print(dictionary[emo])\n\n    img_list = os.listdir(cwd + "/cleaned_images/" + dictionary[emo])\n\n    # same scale\n    scale_percent = 100\n\n    for i in range(len(img_list)):\n        if not os.path.exists("cleaned_images/" + dictionary[emo] + "/" + img_list[i]):\n            continue\n\n        img1 = cv2.imread("cleaned_images/" + dictionary[emo] + "/" + img_list[i])\n        \n        # set size\n        width = int(img1.shape[1] * scale_percent / 100)\n        height = int(img1.shape[0] * scale_percent / 100)\n        dim = (width, height)\n        \n        # resize for comparation\n        img1 = cv2.resize(img1, dim, interpolation = cv2.INTER_AREA)\n\n        # convert the images to grayscale\n        img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)\n        \n        for j in range(i+1, len(im

In [10]:
len(all_image)

24113

In [11]:
# export proprocessed data to preprocessing_data.csv
new_df = pd.DataFrame(all_image, columns=["image", "emotion"])
new_df.to_csv(cwd + "/data_csv/preprocessing_data.csv", index=False)

In [12]:
new_df

,image,emotion
0,facial-expressions_2868585k.jpg,7
1,facial-expressions_2868582k.jpg,3
2,Aaron_Peirsol_0003.jpg,4
3,Aaron_Sorkin_0002.jpg,4
4,Abdel_Nasser_Assidi_0002.jpg,5
...,...,...
24108,SURPRISE/surprise (971).png,7
24109,SURPRISE/surprise (978).jpg,7
24110,SURPRISE/surprise (981).png,7
24111,SURPRISE/surprise (983).jpg,7


In [13]:
new_df.groupby('emotion').count()

,image
emotion,
0,2521
1,1679
2,1568
3,2001
4,6396
5,5383
6,2273
7,2292
